### Drive Mount

In [1]:
from utils.dataloader import DataPartitions, DataGenerator
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [2]:
partitions = DataPartitions(
    past_frames=4, 
    future_frames=4, 
    root="../datasets/arda/04_21_full/",
    partial = 0.3
)

In [3]:
dataset = DataGenerator(
    root="../datasets/arda/04_21_full/", 
    filenames=partitions.get_areas(), 
    dataset_partitions=partitions.get_train(), 
    past_frames=partitions.past_frames, 
    future_frames=partitions.future_frames, 
    input_dim=(partitions.past_frames, 256, 256, 3),  
    output_dim=(partitions.future_frames, 256, 256, 1), 
    batch_size=16, 
    n_channels=1, 
    shuffle=True,
    buffer_size = 1e3,
    buffer_memory = 100
)

In [4]:
X = dataset.get_X()
Y = dataset.get_Y()

X[X > 10e5] = 0 
Y[Y > 10e5] = 0 

100%|██████████| 14/14 [00:26<00:00,  1.90s/it]


In [5]:
X[X > 10e5] = 0 
Y[Y > 10e5] = 0 

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [7]:
sc_img = StandardScaler() # image
sc_vvx = StandardScaler() # vvx
sc_vvy = StandardScaler() # vvy

for sample in X_train:
    for batch in sample:
        for frame in batch:
            sc_img.partial_fit(frame[:,:,0])
            sc_vvx.partial_fit(frame[:,:,1])
            sc_vvy.partial_fit(frame[:,:,1])

In [8]:
for s, sample in enumerate(X_train):
    for b, batch in enumerate(sample):
        for f, frame in enumerate(batch):
            X_train[s, b, f, :, :, 0] = sc_img.transform(frame[:,:,0])
            X_train[s, b, f, :, :, 1] = sc_vvx.transform(frame[:,:,1])
            X_train[s, b, f, :, :, 2] = sc_vvy.transform(frame[:,:,2])
            
print("X_train ready")        

for s, sample in enumerate(X_test):
    for b, batch in enumerate(sample):
        for f, frame in enumerate(batch):
            X_test[s, b, f, :, :, 0] = sc_img.transform(frame[:,:,0])
            X_test[s, b, f, :, :, 1] = sc_vvx.transform(frame[:,:,1])
            X_test[s, b, f, :, :, 2] = sc_vvy.transform(frame[:,:,2])
            
print("X_test transformed")
            
for s, sample in enumerate(y_train):
    for b, batch in enumerate(sample):
        for f, frame in enumerate(batch):
            y_train[s, b, f, :, :, 0] = sc_img.transform(frame[:,:,0])
  
print("y_train transformed")

for s, sample in enumerate(y_test):
    for b, batch in enumerate(sample):
        for f, frame in enumerate(batch):
            y_test[s, b, f, :, :, 0] = sc_img.transform(frame[:,:,0])
            
print("y_test transformed")

X_train ready
X_test transformed
y_train transformed
y_test transformed


<hr>


In [9]:
import numpy as np
import os
from pathlib import Path
import tqdm
import matplotlib.pyplot as plt
import random
import tqdm 

%matplotlib inline

In [10]:
if tf.test.gpu_device_name(): 

    print('Default GPU Device:{}'.format(tf.test.gpu_device_name()))

else:

   print("Please install GPU version of TF")

NameError: name 'tf' is not defined

In [ ]:
class LossHistory(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.losses = []
        
    def loss_plot(self):
        plt.plot(range(len(self.losses)), self.losses)

    def mean_loss(self):
        return np.mean(self.losses)

    def on_batch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))

        #print("Loss: {}, Val_loss: {}, Accuracy: {}".format(logs.get('loss'), logs.get('loss'), logs.get('accuracy')))


In [14]:
tf.__version__

'2.4.1'

### Model

In [41]:
from tensorflow.keras import regularizers
from tensorflow.keras import initializers
from tensorflow.keras import layers
from tensorflow import keras

In [76]:
# https://towardsdatascience.com/residual-blocks-building-blocks-of-resnet-fd90ca15d6ec
# weight - bn - relu - weight - bn- sum - relu
class ResBlock(layers.Layer):
    def __init__(self, channels, kernel_size, depth = 1, padding="same"):
        super(ResBlock, self).__init__()
        self.depth = depth
        self.channels = channels
        self.kernel_size = kernel_size
        self.padding = padding
        
        self.conv1 = layers.Conv3D(self.channels, kernel_size=self.kernel_size,strides=1, padding=self.padding)
        self.conv2 = layers.Conv3D(self.channels, kernel_size=self.kernel_size,strides=1, padding=self.padding)
        self.bn = layers.BatchNormalization()
        self.add = layers.Add()
        self.activation = ReLU()

    def call(self, x):
        for i in range(self.depth): 
            y = self.conv1(x)    
            y = self.bn(y)   
            y = self.activation(y) 
            y = self.conv1(y) 
            y = self.bn(y) 
            x = self.add([x, y]) 
            x = self.activation(x)   
            
        return x

In [77]:
class ResNet(tf.keras.Model):

    def __init__(self):
        super(ResNet, self).__init__()
        
        self.conv1 = layers.Conv3D(6, kernel_size=3,strides=1, padding="same")
        self.block_1 = ResBlock(channels = 6, kernel_size = 3)
        self.pool1 = layers.AveragePooling3D(pool_size=2)
        
        self.conv2 = layers.Conv3D(9, kernel_size=3,strides=1, padding="same")
        self.block_2 = ResBlock(channels = 9, kernel_size = 3)
        self.pool2 = layers.AveragePooling3D(pool_size=2)
        
        self.ups1 = layers.UpSampling3D((2,2,2))
        self.deconv1 = layers.Conv3D(6, kernel_size=3, padding='same')

        self.ups2 = layers.UpSampling3D((2,2,2))
        self.deconv2 = layers.Conv3D(1, kernel_size=3, padding='same')

    def call(self, inputs):
        
        x = self.conv1(inputs)
        x = self.block_1(x)
        x = self.pool1(x)
        
        x = self.conv2(x)
        x = self.block_2(x)
        x = self.pool2(x)
        
        x = self.ups1(x)
        x = self.deconv1(x)
        
        x = self.ups2(x)
        x = self.deconv2(x)
        
        return x

In [78]:
sample_shape = (None, 4, 256, 256, 3)
dropout_rate = 0.5
l1_reg = 1e-3
initializer = initializers.Orthogonal()

In [79]:
resnet = ResNet()

In [80]:
loss = tf.keras.losses.BinaryCrossentropy(
    from_logits=False
)

loss2 = tf.keras.losses.MeanSquaredError()

#, clipvalue=1
resnet.compile(optimizer = Adam(learning_rate = 1e-3), loss = loss2)

In [ ]:
for epoch in range(50):
    print("\n-- Epoch {}".format(epoch))
    print("\t\t Training")
    for i, x in (enumerate(X_train)):
        resnet.fit(
            x = x,
            y = y_train[i],
            callbacks=[history],
            verbose = 0
        )
        
    print("Mean loss: {}".format(history.mean_loss()))
        
    print("\t\t Evaluation ")
    for i, x in (enumerate(X_test)):
        e_lss, _ = resnet.evaluate(
            x = x,
            y = y_test[i],
            verbose = 0
        )
    print("Last eval loss: {}".format(e_lss))


-- Epoch 0
		 Training
Mean loss: 12.52878189086914
		 Evaluation 


In [40]:
# (batch_size, channels, depth, height, width).


x = inputs

# Causes of explosion:
# - AvgPool
# - BatchNorm


#x = BatchNormalization()(x)
#x = Conv3D(6, kernel_size=(3, 3, 3),strides=1, padding='same')(x)
#x = Conv3D(6, kernel_size=(3, 3, 3),strides=1, padding='same')(x)
x = Conv3D(6, kernel_size=3,strides=1, padding="same")(x)

x = ResBlock(channels = 6, kernel_size = 3)(x)
x = AveragePooling3D(pool_size=(2, 2, 2))(x)

x = Conv3D(9, kernel_size=3,strides=1, padding="same")(x)
x = ResBlock(channels = 9, kernel_size = 3)(x)
x = AveragePooling3D(pool_size=(2, 2, 2))(x)

x = UpSampling3D((2,2,2))(x)
x = Conv3D(6, kernel_size=(3, 3, 3), padding='same')(x)
x = Conv3D(6, kernel_size=(3, 3, 3), padding='same')(x)

x = UpSampling3D((2,2,2))(x)
x = Conv3D(1, kernel_size=(3, 3, 3), padding='same')(x)


'''
x = Conv3D(6, kernel_size=(3, 3, 3), padding='same', activation='relu', input_shape=sample_shape,
        kernel_initializer=initializer,
            kernel_regularizer=regularizers.l1( l=l1_reg))(x)

x = BatchNormalization()(x)
x = AveragePooling3D(pool_size=(2, 2, 2))(x)
#x = Dropout(dropout_rate)(x)


x = Conv3D(9, kernel_size=(3, 3, 3), padding='same', activation='relu',
        kernel_initializer=initializer,
           kernel_regularizer=regularizers.l1( l=l1_reg))(x)
x = BatchNormalization()(x)
x = AveragePooling3D(pool_size=(2, 2, 2))(x)
#x = Dropout(dropout_rate)(x)

#x = Conv3DTranspose(filters=9, kernel_size=(3,3,3), strides=(2,2,2), padding="same",activation='relu',
#        kernel_initializer=initializer,
#            kernel_regularizer=regularizers.l1( l=l1_reg))(x)
x = UpSampling3D((2,2,2))(x)
x = BatchNormalization()(x)

#x = Conv3DTranspose(filters=6, kernel_size=(3,3,3), strides=(1,2,2), padding="same",activation='relu',
#        kernel_initializer=initializer,
#            kernel_regularizer=regularizers.l1( l=l1_reg))(x)
x = UpSampling3D((1,2,2))(x)
x = BatchNormalization()(x)

x = Conv3D(1, kernel_size=(3, 3, 3), padding='same',
        kernel_initializer=initializer,
           kernel_regularizer=regularizers.l1( l=l1_reg))(x)
'''

model = Model(inputs, x)
model.summary()

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 4, 256, 256, 3)]  0         
_________________________________________________________________
conv3d_26 (Conv3D)           (None, 4, 256, 256, 6)    492       
_________________________________________________________________
res_block_11 (ResBlock)      (None, 4, 256, 256, 6)    0         
_________________________________________________________________
average_pooling3d_10 (Averag (None, 2, 128, 128, 6)    0         
_________________________________________________________________
conv3d_27 (Conv3D)           (None, 2, 128, 128, 9)    1467      
_________________________________________________________________
res_block_12 (ResBlock)      (None, 2, 128, 128, 9)    0         
_________________________________________________________________
average_pooling3d_11 (Averag (None, 1, 64, 64, 9)      0   

In [36]:
loss = tf.keras.losses.BinaryCrossentropy(
    from_logits=False
)

loss2 = tf.keras.losses.MeanSquaredError()

#, clipvalue=1
model.compile(optimizer = Adam(learning_rate = 1e-3), loss = loss2, metrics = ["accuracy"])
#model.compile(optimizer = Adam(learning_rate = 5e-4), loss = "mean_absolute_error", metrics = ["accuracy"])
#model.compile(optimizer = Adam(learning_rate = 1e-4), loss = "mean_squared_error", metrics = ["accuracy"])

In [37]:
history = LossHistory()

In [38]:
for epoch in range(50):
    print("\n-- Epoch {}".format(epoch))
    print("\t\t Training")
    for i, x in (enumerate(X_train)):
        model.fit(
            x = x,
            y = y_train[i],
            callbacks=[history],
            verbose = 0
        )
        
    print("Mean loss: {}".format(history.mean_loss()))
        
    print("\t\t Evaluation ")
    for i, x in (enumerate(X_test)):
        e_lss, _ = model.evaluate(
            x = x,
            y = y_test[i],
            verbose = 0
        )
    print("Last eval loss: {}".format(e_lss))


-- Epoch 0
		 Training


ValueError: in user code:

    /home/diego/anaconda3/envs/parflood/lib/python3.6/site-packages/tensorflow/python/keras/engine/training.py:805 train_function  *
        return step_function(self, iterator)
    <ipython-input-16-d7180ea86265>:13 call  *
        y = Conv3D(self.channels, kernel_size=self.kernel_size,strides=1, padding=self.padding)(x)
    /home/diego/anaconda3/envs/parflood/lib/python3.6/site-packages/tensorflow/python/keras/engine/base_layer.py:1008 __call__  **
        self._maybe_build(inputs)
    /home/diego/anaconda3/envs/parflood/lib/python3.6/site-packages/tensorflow/python/keras/engine/base_layer.py:2710 _maybe_build
        self.build(input_shapes)  # pylint:disable=not-callable
    /home/diego/anaconda3/envs/parflood/lib/python3.6/site-packages/tensorflow/python/keras/layers/convolutional.py:205 build
        dtype=self.dtype)
    /home/diego/anaconda3/envs/parflood/lib/python3.6/site-packages/tensorflow/python/keras/engine/base_layer.py:639 add_weight
        c

In [ ]:
import random
ind = random.randint(0, X_test.shape[0]-1)

pred = model.predict(X_test[ind])
plt.matshow(pred[0, 0, :, :, 0])
plt.matshow(pred[0, 1, :, :, 0])

In [ ]:
predictions = model.predict(train_dataset.__getitem__(20)[0])[0]
true = test_dataset.__getitem__(20)[1][0]

In [ ]:
fig, axs = plt.subplots(1,partitions.future_frames, figsize=(15,5))

for i, frame in enumerate(predictions):
    axs[i].matshow(frame.reshape(256, 256))

In [ ]:
fig, axs = plt.subplots(1,3, figsize=(15,5))

for i, frame in enumerate(true):
    axs[i].matshow(frame.reshape(256, 256))

In [ ]:
plt.plot(range(4), [np.mean(x) for x in predictions])
plt.plot(range(4), [np.mean(x) for x in true])

In [ ]:
ind = 6

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15,5))
fig.suptitle('frame 1 - frame 11 pred - frame 11 true')

ax1.matshow(train_d.__getitem__(ind)[0][0][0].reshape(256,256))

ax2.matshow(model.predict(train_d.__getitem__(ind)[0])[0][0].reshape(256,256))

ax3.matshow(train_d.__getitem__(ind)[1][0][0].reshape(256,256))

In [ ]:
f = np.concatenate((train_d.__getitem__(ind)[0][0], train_d.__getitem__(ind)[1][0]))
plt.plot(range(14), [np.mean(x) for x in f])

plt.plot(range(10, 14), [np.mean(x) for x in model.predict(train_d.__getitem__(ind)[0])[0]])

In [ ]:
history.accuracy_plot()

In [ ]:
history.loss_plot()